Author: Lani Cupo       Date: 20190930
This script uses a "pilot" to simulate data for a power analysis using the simr package in R. 
Tutorials used include: 
https://rstudio-pubs-static.s3.amazonaws.com/484106_6b51212f20164fdd88cd7cce89bdef79.html
https://cran.r-project.org/web/packages/simr/vignettes/examples.html


In [1]:
#load all necessary libraries
#On CIC computers, simr is installed in R 3.5.1
library(lme4)
library(dplyr)
library(ggplot2)
library(simr)
library(broom) #extract random effects
library(knitr)
library(truncnorm)
library(lmerTest)
library(RMINC)

Loading required package: Matrix

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



ERROR: Error in library(simr): there is no package called ‘simr’


In [ ]:
#set working directory
setwd("/home/lani/phd1/pwr_analysis")

For this simulation:
my outcome (y) <- brain volume (of specific region)
group level predictors (fixed effects) <- treatment + sex + age
individual level predictors (random effects) <- mouse_id

In [ ]:
#load in data sets
volumes <- read.csv("allvolumes_mia.csv")


In [ ]:
#Alter timepoints to look at the adolescent time points (that I have in my experiment)
vols_adol <- subset(volumes, !volumes$timepoint == 0)
vols_adol <- subset(vols_adol, !vols_adol$timepoint == 3)
#Exclude the poly late group which I don't have
vols_adol_e <- subset(vols_adol, !vols_adol$new_groups == "POL_L")

In [ ]:
#run lmer that you want to look at 
#Here I chose one region to look at first
fm1 <- lmer(left_CA1Rad ~ sex + new_groups + age + (1|mouse_id), data = vols_adol_e)
summary(fm1)

In [ ]:
#extract effects from the model to base simulations on
#extract fixed
fixed <- c(1.427, 0.054, -0.073, 0.004)
#extract random effects
rand <- 0.002
#extract residual and take the square root
s <- 0.004^0.5

In [ ]:
#recreate the model with the coefficients from before
model <- makeLmer(y ~ sex + new_groups + age + (1|mouse_id), fixef=fixed, VarCorr=rand, sigma=s, data=vols_adol_e)
model

In [ ]:
#Test a different sample size (here n=500)
# variable identified is expaneded to n rows 
model_ext_mice <- extend(model, along="mouse_id", n=500)
model_ext_mice

In [ ]:
#Test 2-way interactions
set.seed(0)
p_curve_treatXsex <- powerCurve(model_ext_mice, 
                          test=fcompare(y ~ new_groups + sex), 
                          along="mouse_id",
                          breaks=c(1,2,3,4,5,6,7,8,9),
                          nsim=500,alpha=.05, progress=FALSE)
plot(p_curve_treatXsex)


#Test 2-way interactions
set.seed(0)
p_curve_treatXage <- powerCurve(model_ext_mice, 
                                test=fcompare(y ~ new_groups + age), 
                                along="mouse_id",
                                breaks=c(1,10, 20, 30, 40, 50),
                                nsim=500,alpha=.05, progress=FALSE)
plot(p_curve_treatXage)

#Test 2-way interactions
set.seed(0)
p_curve_sexXage <- powerCurve(model_ext_mice, 
                                test=fcompare(y ~ sex + age), 
                                along="mouse_id",
                                breaks=c(1,10, 20, 30, 40, 50),
                                nsim=500,alpha=.05, progress=FALSE)
plot(p_curve_sexXage)


In [ ]:
#Test 3-way interaction
set.seed(0)
p_curve_ixn <- powerCurve(model_ext_mice, 
                             test=fcompare(y ~ new_groups + sex + age), 
                             along="mouse_id",
                             breaks=c(50, 100, 150, 200),
                             nsim=500,alpha=.05, progress=FALSE)
plot(p_curve_ixn)

Now, we will attempt another method for estimating power, given different numbers of inputs. 

In [ ]:
#Calculate p-values for individual terms like this:
summary(fm1)$coef
doTest(fm1, fixed("new_groupsSAL", "z"))
doTest(fm1, fixed("age", "z"))
doTest(fm1, fixed("sexmale", "z"))

In [ ]:
#Simulate and test for power
#Set the effect size for the fixed effects based on our Hedge's g calculation as it represents
#the smallest effect we can reliably detect.
fixef(fm1)["new_groupsSAL"] <- 0.267
sim_grp <- powerSim(fm1, fixed("new_groupsSAL", "z"), nsim=50)

fixef(fm1)["age"] <- 0.267
powerSim(fm1, fixed("age", "z"), nsim=50)

fixef(fm1)["sexmale"] <- 0.267
sim_sex <- powerSim(fm1, fixed("sexmale", "z"), nsim=50)